In [47]:

const wave_lenght = 1550 # nm
const wave_length_m = wave_lenght * 1e-9 # m

const distance_btw_cities = 78.5 # km
const distance_btw_cities_m = distance_btw_cities * 1e3 # m

const As = 0.4 # dB
const Ac = 0.4 # dB
# Distance between welds
const LX = 15.0 # km
const maring = 4.0 # db

mutable struct Fiber
    name::String
    damping_factor::Float64
end

# Array of capacities 1Gbit/s, 5Gbit/s, 50Gbit/s
const capacities = [1e9, 5e9, 50e9]
const f = 193.1e12 # 1 GHz
const h  = 6.626e-34  # stała Plancka

# Create a object for the fiber
fiber_smf = Fiber("SMF", 0.25)
fiber_sm1 = Fiber("SM1", 0.7)

Fiber("SM1", 0.7)

In [48]:
# power budget counting function
function calculate_power_budget(
  α::Float64,  # współczynnik tłumienia światłowodu
  L::Float64,  # długość łącza
  As::Float64, # tłumienie spawu
  Ls::Float64, # odległość między spawami
  X::Float64,  # straty wniesione przez siatkę Bragga lub światłowód kompensujący
  n::Float64,  # liczba złączek
  Ac::Float64, # tłumienie złączki
  M::Float64   # margines na starzenie się łącza
)
  fiber_loss = α * L
  splice_loss = As * (L / Ls - 1)
  connector_loss = n * Ac

  G = fiber_loss + splice_loss + X + connector_loss + M

  return G
end

calculate_power_budget (generic function with 1 method)

In [49]:
const h = 6.62e-34

function calculate_photodiode_power(
  n0::Float64,  # liczba fotonów/bit
  f::Float64,   # częstotliwość [Hz]
  B0::Float64   # szybkość transmisji [bit/s]
)
  # Obliczanie mocy według wzoru: PR[dBm] = 10 log(n0·h·f·B0 / 1e-3)
  power_linear = (n0 * h * f * B0) / 1e-3
  power_dbm = 10 * log10(power_linear) + 3.0
  return power_dbm 
end

calculate_photodiode_power (generic function with 1 method)

In [50]:
using Plots

function find_optimal_n0(x, y)
  dy = diff(y) ./ y[1:end-1]

  threshold = 0.01 
  optimal_idx = findfirst(abs.(dy) .< threshold)

  if isnothing(optimal_idx)
    return 200.0
  end

  return x[optimal_idx]
end

x = 100.0:10.0:2000.0

for capacity in capacities
  y_smf = Float64[]
  y_sm1 = Float64[]

  for n0 in x
    local photodiode_power = calculate_photodiode_power(n0, f, capacity)

    local power_budget = calculate_power_budget(fiber_smf.damping_factor, distance_btw_cities, As, LX, 0.0, 2.0, Ac, maring)
    local min_power_budget_smf = power_budget + photodiode_power
    push!(y_smf, min_power_budget_smf)

    local power_budget = calculate_power_budget(fiber_sm1.damping_factor, distance_btw_cities, As, LX, 0.0, 2.0, Ac, maring)
    local min_power_budget_sm1 = power_budget + photodiode_power
    push!(y_sm1, min_power_budget_sm1)
  end

  local optimal_n0_smf = find_optimal_n0(x, y_smf)
  local optimal_n0_sm1 = find_optimal_n0(x, y_sm1)


  local optimal_idx_smf = findfirst(x .== optimal_n0_smf)
  local optimal_idx_sm1 = findfirst(x .== optimal_n0_sm1)

  scatter!([optimal_n0_smf], [y_smf[optimal_idx_smf]],
    label="Optimal SMF ($(round(optimal_n0_smf)) fotons/bit)",
    marker=:star)
  scatter!([optimal_n0_sm1], [y_sm1[optimal_idx_sm1]],
    label="Optimal SM1 ($(round(optimal_n0_sm1)) fotons/bit)",
    marker=:star)

  println("\nFor capacity $(capacity/1e9) Gbit/s:")
  println("Optimal n0 for SMF: $(round(optimal_n0_smf)) fotons/bit")
  println("Optimal n0 for SM1: $(round(optimal_n0_sm1)) fotons/bit")

end


For capacity 1.0 Gbit/s:
Optimal n0 for SMF: 280.0 fotons/bit
Optimal n0 for SM1: 230.0 fotons/bit

For capacity 5.0 Gbit/s:
Optimal n0 for SMF: 200.0 fotons/bit
Optimal n0 for SM1: 180.0 fotons/bit

For capacity 50.0 Gbit/s:
Optimal n0 for SMF: 740.0 fotons/bit
Optimal n0 for SM1: 130.0 fotons/bit


In [59]:
using PrettyTables
using Plots


function get_optimal_n0(capacity, fiber_type)
  if fiber_type == "SMF"
    if capacity == 1e9
      return 280.0
    elseif capacity == 5e9
      return 200.0
    else  # 50e9
      return 740.0
    end
  else  # SM1
    if capacity == 1e9
      return 230.0
    elseif capacity == 5e9
      return 180.0
    else  # 50e9
      return 130.0
    end
  end
end

println("SMF fiber")
results_smf = Matrix{Float64}(undef, length(capacities), 5)

for (i, capacity) in enumerate(capacities)
  local n0_optimal = get_optimal_n0(capacity, "SMF")
  local photodiode_power = calculate_photodiode_power(n0_optimal, f, capacity)
  local power_budget = calculate_power_budget(fiber_smf.damping_factor, distance_btw_cities, As, LX, 0.0, 2.0, Ac, maring)
  local min_power_budget = power_budget + photodiode_power
  results_smf[i, :] = [capacity / 1e9, photodiode_power, power_budget, min_power_budget, 10^(min_power_budget / 10)]
end

const smf1b = results_smf[1, 4]
const smf5b = results_smf[2, 4]
const smf50b = results_smf[3, 4]


println("\nSM1 fiber")
results_sm1 = Matrix{Float64}(undef, length(capacities), 5)

for (i, capacity) in enumerate(capacities)
  local n0_optimal = get_optimal_n0(capacity, "SM1")
  local photodiode_power = calculate_photodiode_power(n0_optimal, f, capacity)
  local power_budget = calculate_power_budget(fiber_sm1.damping_factor, distance_btw_cities, As, LX, 0.0, 2.0, Ac, maring)
  local min_power_budget = power_budget + photodiode_power
  results_sm1[i, :] = [capacity / 1e9, photodiode_power, power_budget, min_power_budget, 10^(min_power_budget / 10)]
end

const sm1_1b = results_sm1[1, 4]
const sm1_5b = results_sm1[2, 4]
const sm1_50b = results_sm1[3, 4]

############################### Tworzenie wykresów ########################################
x = 100.0:10.0:2000.0

for capacity in capacities
  y_smf = Float64[]
  y_sm1 = Float64[]
  y_smf_mW = Float64[]
  y_sm1_mW = Float64[]

  for n0 in x
    # SMF calculations
    photodiode_power = calculate_photodiode_power(n0, f, capacity)
    power_budget = calculate_power_budget(fiber_smf.damping_factor, distance_btw_cities, As, LX, 0.0, 2.0, Ac, maring)
    min_power_budget = power_budget + photodiode_power
    push!(y_smf, min_power_budget)
    push!(y_smf_mW, 10^(min_power_budget / 10))

    # SM1 calculations
    power_budget = calculate_power_budget(fiber_sm1.damping_factor, distance_btw_cities, As, LX, 0.0, 2.0, Ac, maring)
    min_power_budget = power_budget + photodiode_power
    push!(y_sm1, min_power_budget)
    push!(y_sm1_mW, 10^(min_power_budget / 10))
  end

  # Create and save dBm plot
  p1 = plot(x, y_smf,
    label="SMF",
    xlabel="Fotons/bit",
    ylabel="Minimum transmitter power (dBm)",
    title="SMF and SM1 fibers for $(capacity/1e9) Gbit/s capacity")
  plot!(p1, x, y_sm1, label="SM1")
  savefig(p1, "power_budget_$(capacity/1e9)Gbit_dBm.png")

  # Create and save mW plot
  p2 = plot(x, y_smf_mW,
    label="SMF",
    xlabel="Fotons/bit",
    ylabel="Minimum transmitter power (mW)",
    title="SMF and SM1 fibers for $(capacity/1e9) Gbit/s capacity (mW)")
  plot!(p2, x, y_sm1_mW, label="SM1")
  savefig(p2, "power_budget_$(capacity/1e9)Gbit_mW.png")
end

############################### Wyświetlanie tabel ########################################

header = (
  ["Capacity", "Photodiode Power", "Power Budget", "Min Transmitter Power", "Min Transmitter Power mW"],
  ["[Gbit/s]", "[dBm]", "[dB]", "[dB]", "[mW]"]
)

hl_positive = Highlighter(
  (data, i, j) -> data[i, j] > 0,
  crayon"green bold"
)

hl_negative = Highlighter(
  (data, i, j) -> data[i, j] < 0,
  crayon"red bold"
)

println("\nResults for SMF fiber (optimal n0):")
pretty_table(
  results_smf;
  formatters=ft_printf("%8.3f", 1:5),
  header=header,
  header_crayon=crayon"yellow bold",
  highlighters=(hl_positive, hl_negative),
  tf=tf_unicode_rounded
)

println("\nResults for SM1 fiber (optimal n0):")
pretty_table(
  results_sm1;
  formatters=ft_printf("%8.3f", 1:5),
  header=header,
  header_crayon=crayon"yellow bold",
  highlighters=(hl_positive, hl_negative),
  tf=tf_unicode_rounded
)

SMF fiber

SM1 fiber

Results for SMF fiber (optimal n0):
╭──────────┬──────────────────┬──────────────┬───────────────────────┬──────────────────────────╮
│ Capacity │ Photodiode Power │ Power Budget │ Min Transmitter Power │ Min Transmitter Power mW │
│ [Gbit/s] │            [dBm] │         [dB] │                  [dB] │                     [mW] │
├──────────┼──────────────────┼──────────────┼───────────────────────┼──────────────────────────┤
│    1.000 │          -41.462 │       26.118 │               -15.344 │                    0.029 │
│    5.000 │          -35.934 │       26.118 │                -9.815 │                    0.104 │
│   50.000 │          -20.252 │       26.118 │                 5.867 │                    3.861 │
╰──────────┴──────────────────┴──────────────┴───────────────────────┴──────────────────────────╯

Results for SM1 fiber (optimal n0):
╭──────────┬──────────────────┬──────────────┬───────────────────────┬──────────────────────────╮
│ Capacity │ Photodiode

In [56]:
function calculate_L0(α, Ps, Pc, Pm, n0, h, f)
    return (1 / α) * (Ps - Pc - Pm - 30 - 10 * log10(n0 * h * f))
end

function calculate_L(L0, α, B0)
    return L0 - (10 / α) * log10(B0)
end

# Przygotuj dane do wykresu
B0_range = 1e9:1e9:50e9  # od 1G do 50G

# Oblicz Pc (straty na połączeniach)
Pc_smf = As + LX + 0.0 + 2.0 * Ac
Pc_sm1 = Pc_smf

# Przygotuj tablice na wyniki
L_smf = Float64[]
L_sm1 = Float64[]

# Obliczenia dla SMF
for B0 in B0_range
    local n0 = 200.0
    local Ps = smf50b
    local L0 = calculate_L0(fiber_smf.damping_factor, Ps, Pc_smf, maring, n0, h, f)
    local L = calculate_L(L0, fiber_smf.damping_factor, B0)
    push!(L_smf, L)
end

# Obliczenia dla SM1
for B0 in B0_range
    local n0 = 200.0
    local Ps = sm1_50b
    local L0 = calculate_L0(fiber_sm1.damping_factor, Ps, Pc_sm1, maring, n0, h, f)
    local L = calculate_L(L0, fiber_sm1.damping_factor, B0) 
    push!(L_sm1, L)
end

# Stwórz wykresy
p1 = plot(
    B0_range ./ 1e9,
    L_smf,
    label="SMF (α = $(fiber_smf.damping_factor) dB/km)",
    xlabel="Przepływność [Gbit/s]",
    ylabel="Zasięg łącza [km]",
    title="Zasięg łącza SMF w funkcji przepływności",
    linewidth=2,
    legend=:topright
)

# Zaznacz punkty charakterystyczne dla SMF
specific_points = [1e9, 5e9, 50e9]
smf_points = Float64[]

for B0 in specific_points
    idx = findfirst(B0_range .== B0)
    push!(smf_points, L_smf[idx])
end

scatter!(
    p1,
    specific_points ./ 1e9,
    smf_points,
    label="Punkty charakterystyczne",
    marker=:circle
)

# Dodaj wartości dla SMF
for (i, B0) in enumerate(specific_points)
    annotate!(
        p1,
        B0 / 1e9,
        smf_points[i],
        text("$(round(smf_points[i], digits=1)) km", :bottom, 8)
    )
end

# Wykres dla SM1
p2 = plot(
    B0_range ./ 1e9,
    L_sm1,
    label="SM1 (α = $(fiber_sm1.damping_factor) dB/km)",
    xlabel="Przepływność [Gbit/s]",
    ylabel="Zasięg łącza [km]",
    title="Zasięg łącza SM1 w funkcji przepływności",
    linewidth=2,
    legend=:topright
)

# Zaznacz punkty charakterystyczne dla SM1
sm1_points = Float64[]

for B0 in specific_points
    idx = findfirst(B0_range .== B0)
    push!(sm1_points, L_sm1[idx])
end

scatter!(
    p2,
    specific_points ./ 1e9,
    sm1_points,
    label="Punkty charakterystyczne",
    marker=:circle
)

# Dodaj wartości dla SM1
for (i, B0) in enumerate(specific_points)
    annotate!(
        p2,
        B0 / 1e9,
        sm1_points[i],
        text("$(round(sm1_points[i], digits=1)) km", :bottom, 8)
    )
end

# Połącz wykresy w jeden układ
plot(p1, p2, layout=(2, 1), size=(800, 1000))
savefig("link_range_vs_capacity_separate.png")

# Wydrukuj wartości
println("\nZasięg łącza dla charakterystycznych przepływności:")
println("\nSMF:")
for (i, B0) in enumerate(specific_points)
    println("$(B0/1e9) Gbit/s: $(round(smf_points[i], digits=2)) km")
end
println("\nSM1:")
for (i, B0) in enumerate(specific_points)
    println("$(B0/1e9) Gbit/s: $(round(sm1_points[i], digits=2)) km")
end


Zasięg łącza dla charakterystycznych przepływności:

SMF:
1.0 Gbit/s: 126.36 km
5.0 Gbit/s: 98.4 km
50.0 Gbit/s: 58.4 km

SM1:
1.0 Gbit/s: 84.8 km
5.0 Gbit/s: 74.82 km
50.0 Gbit/s: 60.53 km
